# Set up Overmind

Set api keys as env variables. Alternatively they can be passed directly to the client.

In [6]:
import os
from overmind.client import OvermindClient

os.environ["OVERMIND_API_KEY"] = "your_overmind_api_key"
os.environ["OPENAI_API_KEY"] = "your_openai_api_key"

overmind = OvermindClient()

Overmind Proxy client is a drop in replacement for any LLM client we support (currently OpenAI, more to come soon).

The client accepts all parameters normally required by your LLM vendor + a list of input and output policies, as in the example below.

# Basic usage with responses API & MCP

In [44]:
input_messages = [
    {"role": "user", "content": "Tell me 1 sentence joke about Gary, who lives in London and likes birds"}
]

input_messages = "Tell me 1 sentence joke about Gary, who lives in London and likes birds"

result = overmind.openai.responses.create(
    model='gpt-5-mini',
    input=input_messages,
    # Overmind built-in policies
    input_policies=['anonymize_pii'],
    output_policies=['reject_irrelevant_answer'],
)

result.summary()

╭─────────────────────────────────────────────── Proxy Run Summary ───────────────────────────────────────────────╮
│           Results                                                                                               │
│ ┏━━━━━━━━━━━━━━┳━━━━━━━━━━━┓                                                                                    │
│ ┃ Component    ┃  Outcome  ┃                                                                                    │
│ ┡━━━━━━━━━━━━━━╇━━━━━━━━━━━┩                                                                                    │
│ │ Input Layer  │ ⚠️ ALTERED │                                                                                    │
│ │ Output Layer │ ✅ PASSED │                                                                                    │
│ └──────────────┴───────────┘                                                                                    │
│                                                                                                                 │
│                                                 Policy Results                                                  │
│ ┏━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓  │
│ ┃ Layer      ┃ Policy               ┃ Details                                                                ┃  │
│ ┡━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩  │
│ │ In         │ anonymize_pii        │ {                                                                      │  │
│ │            │                      │   "result": "altered",                                                 │  │
│ │            │                      │   "policy_results": {                                                  │  │
│ │            │                      │     "PII_detected": true                                               │  │
│ │            │                      │   }                                                                    │  │
│ │            │                      │ }                                                                      │  │
│ │ Out        │ reject_irrelevant_a… │ {                                                                      │  │
│ │            │                      │   "result": "passed",                                                  │  │
│ │            │                      │   "policy_results": {                                                  │  │
│ │            │                      │     "relevant_answer": true                                            │  │
│ │            │                      │   }                                                                    │  │
│ │            │                      │ }                                                                      │  │
│ └────────────┴──────────────────────┴────────────────────────────────────────────────────────────────────────┘  │
│ ╭─ Processed Input ───────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ "Tell me 1 sentence joke about [PERSON_NAME], who lives in [GEOGRAPHIC_DATA] and likes birds"               │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭─ Processed Output ──────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ "Since [PERSON_NAME] moved to [GEOGRAPHIC_DATA], the local birds started charging rent \u2014 they claim    │ │
│ │ his backyard is prime \"tweet\" real estate."                                                               │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

This flexibility also allows to support OpenAI's native remote MCP usage. Since our input policies are applied before any data is passed to OpenAI, this will also mitigate risk of data exposure to MCP servers.

In [46]:
result = overmind.openai.responses.create(
    model='gpt-5-mini',
    input="""I have a request payload like {"full name": "Jon Smith", "question": "question text"}, how do I send it to ChatGPT
    with openai responses api? Please reference openai repo.""",
    tools=[
        {
            "type": "mcp",
            "server_label": "deepwiki",
            "server_url": "https://mcp.deepwiki.com/mcp",
            "require_approval": "never",
        },
    ],
    # Overmind built-in policies
    input_policies=['anonymize_pii'],
    output_policies=['reject_irrelevant_answer'],
)

result.summary()

╭─────────────────────────────────────────────── Proxy Run Summary ───────────────────────────────────────────────╮
│           Results                                                                                               │
│ ┏━━━━━━━━━━━━━━┳━━━━━━━━━━━┓                                                                                    │
│ ┃ Component    ┃  Outcome  ┃                                                                                    │
│ ┡━━━━━━━━━━━━━━╇━━━━━━━━━━━┩                                                                                    │
│ │ Input Layer  │ ⚠️ ALTERED │                                                                                    │
│ │ Output Layer │ ✅ PASSED │                                                                                    │
│ └──────────────┴───────────┘                                                                                    │
│                                                                                                                 │
│                                                 Policy Results                                                  │
│ ┏━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓  │
│ ┃ Layer      ┃ Policy               ┃ Details                                                                ┃  │
│ ┡━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩  │
│ │ In         │ anonymize_pii        │ {                                                                      │  │
│ │            │                      │   "result": "altered",                                                 │  │
│ │            │                      │   "policy_results": {                                                  │  │
│ │            │                      │     "PII_detected": true                                               │  │
│ │            │                      │   }                                                                    │  │
│ │            │                      │ }                                                                      │  │
│ │ Out        │ reject_irrelevant_a… │ {                                                                      │  │
│ │            │                      │   "result": "passed",                                                  │  │
│ │            │                      │   "policy_results": {                                                  │  │
│ │            │                      │     "relevant_answer": true                                            │  │
│ │            │                      │   }                                                                    │  │
│ │            │                      │ }                                                                      │  │
│ └────────────┴──────────────────────┴────────────────────────────────────────────────────────────────────────┘  │
│ ╭─ Processed Input ───────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ "I have a request payload like {\"full name\": \"[PERSON_NAME]\", \"question\": \"question text\"}, how do  │ │
│ │ I send it to ChatGPT\n    with openai responses api? Please reference openai repo."                         │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭─ Processed Output ──────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ "Short answer: stringify the payload and send it as the Responses API \"input\" (or as an input_text        │ │
│ │ content item). You can also pass an instructions string telling the model to parse the JSON and answer the  │ │
│ │ question. Below are minimal Python and JavaScript examples based on the openai/openai-python and openai     │ │
│ │ (JS) usage patterns for the Responses API.\n\nPython

# Customising Policies

All client calls can be extended with `input_policies` and `output_policies` aruments. The below is the example setting up PII anonymisation and LLM judge. For all current templates see [Policies.md](https://github.com/overmind-core/overmind-python/blob/main/docs/Policies.md)

This set up allows you to experiment with different templates and parameters when you need more customisation.

In [23]:
input_pii_policy = {
    'policy_template': 'anonymize_pii',
    'parameters': {
        'pii_types': ['DEMOGRAPHIC_DATA', 'FINANCIAL_ID']
    }
}

output_llm_judge_criteria = {
    'policy_template': 'reject_llm_judge_with_criteria',
    'parameters': {
        'criteria': [
            "Must be succinct and easy to understand",
        ]
    }
}

input_messages = [
    {"role": "user", "content": "Hi my name is Jon, account number 20194812. Should I switch my mortgage now or wait for a year to have a lower interest rate?"}
]

result = overmind.openai.responses.create(
    model='gpt-5-mini',
    input=input_messages,
    input_policies=[input_pii_policy],
    output_policies=[output_llm_judge_criteria]
)

result.summary()

╭─────────────────────────────────────────────── Proxy Run Summary ───────────────────────────────────────────────╮
│           Results                                                                                               │
│ ┏━━━━━━━━━━━━━━┳━━━━━━━━━━━┓                                                                                    │
│ ┃ Component    ┃  Outcome  ┃                                                                                    │
│ ┡━━━━━━━━━━━━━━╇━━━━━━━━━━━┩                                                                                    │
│ │ Input Layer  │ ⚠️ ALTERED │                                                                                    │
│ │ Output Layer │ ✅ PASSED │                                                                                    │
│ └──────────────┴───────────┘                                                                                    │
│                                                                                                                 │
│                                                 Policy Results                                                  │
│ ┏━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓  │
│ ┃ Layer      ┃ Policy               ┃ Details                                                                ┃  │
│ ┡━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩  │
│ │ In         │ anonymize_pii        │ {                                                                      │  │
│ │            │                      │   "result": "altered",                                                 │  │
│ │            │                      │   "policy_results": {                                                  │  │
│ │            │                      │     "PII_detected": true                                               │  │
│ │            │                      │   }                                                                    │  │
│ │            │                      │ }                                                                      │  │
│ │ Out        │ reject_llm_judge_wi… │ {                                                                      │  │
│ │            │                      │   "result": "passed",                                                  │  │
│ │            │                      │   "policy_results": {                                                  │  │
│ │            │                      │     "complies_with_criteria": true                                     │  │
│ │            │                      │   }                                                                    │  │
│ │            │                      │ }                                                                      │  │
│ └────────────┴──────────────────────┴────────────────────────────────────────────────────────────────────────┘  │
│ ╭─ Processed Input ───────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ [                                                                                                           │ │
│ │   {                                                                                                         │ │
│ │     "role": "user",                                                                                         │ │
│ │     "content": "Hi my name is Jon, account number [FINANCIAL_ID]. Should I switch my mortgage now or wait   │ │
│ │ for a year to have a lower interest rate?"                                                                  │ │
│ │   }                                                                                                         │ │
│ │ ]                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────

The LLM judge policy is very flexible, for example it can block LLM outputs containing financial advice.

In [22]:
input_pii_policy = {
    'policy_template': 'anonymize_pii',
    'parameters': {
        'pii_types': ['DEMOGRAPHIC_DATA', 'FINANCIAL_ID']
    }
}

output_llm_judge_criteria = {
    'policy_template': 'reject_llm_judge_with_criteria',
    'parameters': {
        'criteria': [
            "Must not contain financial advice",
            "Must be succinct and easy to understand",
        ]
    }
}

input_messages = [
    {"role": "user", "content": "Hi my name is Jon, account number 20194812. Should I switch my mortgage now or wait for a year to have a lower interest rate?"}
]

result = overmind.openai.responses.create(
    model='gpt-5-mini',
    input=input_messages,
    input_policies=[input_pii_policy],
    output_policies=[output_llm_judge_criteria]
)

result.summary()

╭─────────────────────────────────────────────── Proxy Run Summary ───────────────────────────────────────────────╮
│            Results                                                                                              │
│ ┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓                                                                                  │
│ ┃ Component    ┃   Outcome   ┃                                                                                  │
│ ┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩                                                                                  │
│ │ Input Layer  │  ⚠️ ALTERED  │                                                                                  │
│ │ Output Layer │ ❌ REJECTED │                                                                                  │
│ └──────────────┴─────────────┘                                                                                  │
│                                                                                                                 │
│                                                 Policy Results                                                  │
│ ┏━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓  │
│ ┃ Layer      ┃ Policy               ┃ Details                                                                ┃  │
│ ┡━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩  │
│ │ In         │ anonymize_pii        │ {                                                                      │  │
│ │            │                      │   "result": "altered",                                                 │  │
│ │            │                      │   "policy_results": {                                                  │  │
│ │            │                      │     "PII_detected": true                                               │  │
│ │            │                      │   }                                                                    │  │
│ │            │                      │ }                                                                      │  │
│ │ Out        │ reject_llm_judge_wi… │ {                                                                      │  │
│ │            │                      │   "result": "rejected",                                                │  │
│ │            │                      │   "policy_results": {                                                  │  │
│ │            │                      │     "complies_with_criteria": false,                                   │  │
│ │            │                      │     "reason": "The response contains financial advice as it provides a │  │
│ │            │                      │ decision framework and calculations for determining whether to switch  │  │
│ │            │                      │ a mortgage, which is a financial decision."                            │  │
│ │            │                      │   }                                                                    │  │
│ │            │                      │ }                                                                      │  │
│ └────────────┴──────────────────────┴────────────────────────────────────────────────────────────────────────┘  │
│ ╭─ Processed Input ───────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ [                                                                                                           │ │
│ │   {                                                                                                         │ │
│ │     "role": "user",                                                                                         │ │
│ │     "content": "Hi my name is Jon, account number [FINANCIAL_ID]. Should I switch my mortgage now or wait   │ │
│ │ for a year to have a lower interest rate?"          

# Getting non-text data

The output of a model doesn't have to be text - for example, let's say we want to use it to generate document embedding for our RAG application, while obfuscating PII data

In [28]:
input_document = """
Dear Jon Smith,

We're glad to inform you that your visa application for passport number 19318291 has been successfull...
"""

result = overmind.openai.embeddings.create(
    model="text-embedding-3-small", 
    input=input_document, 
    encoding_format="float",
    input_policies=['anonymize_pii'],
)

result.summary()

╭─────────────────────────────────────────────── Proxy Run Summary ───────────────────────────────────────────────╮
│           Results                                                                                               │
│ ┏━━━━━━━━━━━━━┳━━━━━━━━━━━┓                                                                                     │
│ ┃ Component   ┃  Outcome  ┃                                                                                     │
│ ┡━━━━━━━━━━━━━╇━━━━━━━━━━━┩                                                                                     │
│ │ Input Layer │ ⚠️ ALTERED │                                                                                     │
│ └─────────────┴───────────┘                                                                                     │
│                                                                                                                 │
│                                                 Policy Results                                                  │
│ ┏━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓  │
│ ┃ Layer      ┃ Policy               ┃ Details                                                                ┃  │
│ ┡━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩  │
│ │ In         │ anonymize_pii        │ {                                                                      │  │
│ │            │                      │   "result": "altered",                                                 │  │
│ │            │                      │   "policy_results": {                                                  │  │
│ │            │                      │     "PII_detected": true                                               │  │
│ │            │                      │   }                                                                    │  │
│ │            │                      │ }                                                                      │  │
│ └────────────┴──────────────────────┴────────────────────────────────────────────────────────────────────────┘  │
│ ╭─ Processed Input ───────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ "\nDear [PERSON_NAME],\n\nWe're glad to inform you that your visa application for passport number           │ │
│ │ [GOVERNMENT_ID] has been successfull...\n"                                                                  │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭─ Processed Output ──────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ [0.02429097, -0.030140068, 0.021721952, 0.009089043, -0.021515513, -0.03169983, -0.021813704, 0.016824765,  │ │
│ │ -0.021389356, -0.035599228, 0.0154714435, -0.0060039293, -0.028121557, -0.016652733, 0.026171856,           │ │
│ │ 0.06757431, -0.00019550761, 0.04408616, -0.03991151, 0.0023224368, 0.048903067, 0.056793615, -0.010516912,  │ │
│ │ 0.004263535, 0.003701563, 0.0022450222, -0.04486604, -0.045714732, 0.004693616, -0.046586365, 0.024359781,  │ │
│ │ -0.01385434, 0.016021946, 0.016549513, -0.0058949753, 0.030346507, -0.031676892, -0.013624962,              │ │
│ │ -0.016228385, -0.03362659, 0.036723174, -0.014898002, 0.037663616, 0.023648715, -0.026171856, 0.0627574,    │ │
│ │ -0.01804046, -0.041861206, 0.0035954763, 0.030552946, 0.0046076, -0.012317517, 0.020036036, 0.10349467,     │ │
│ │ -0.008171537, -0.016010478, 0.062665656, -0.025896605, -0.04738918, -0.03179158, 0.042480525, 0.023534026,  │ │
│ │ 0.01433603, 0.01560907, 0.03208977, 0.021871047, -0.029222563, 0.032823775, 0.06587692, 0.011055946,        │ │
│ │ 0.008572946, 0.09927414, 0.024268031, 0.009639546, -0.0042348634, -0.018499212, -0.009788641, 0.016343074,  │ │
│ │ -0.020517725, -0.00040320083, 0.0133726485, -0.0250